# Preprocess: VOLUME & PERCENT extraction before NER

В этом ноутбуке реализуем каскад предобработки поисковых запросов:
1. **Явное извлечение** `PERCENT` ("%", "процент") и `VOLUME` (юниты: мл/л/г/кг/шт/уп/пак и пр.)
2. **Классификация "голых чисел"** (без указателей) в `PERCENT`/`VOLUME`/`OTHER`:
   - используем **лексикон "жирных" продуктов**, собранный **из train.csv** по совместной встречаемости с процентами;
   - простые **диапазоны** (например, `0 → PERCENT`, числа `1..40` рядом с жирными терминами → `PERCENT`, большие числа → чаще `VOLUME`).

Вы получите функции:
- `extract_explicit_numeric(text)` — явные `%` и объёмы по regex;
- `infer_implicit_numeric(text, fatty_words=...)` — классификация "голых" чисел;
- `preprocess_query(text, fatty_words=...)` — единая предобработка (возвращает сущности и текст с масками).

> **Зачем:** снять из NER конфликтующие числовые сущности заранее (особенно там, где модель путает `BRAND/TYPE` с числами).

In [2]:
from __future__ import annotations
import pandas as pd
import regex as re
from collections import Counter
from typing import List, Tuple

TRAIN_PATH = "data/train.csv"  # подставь свой путь
WORD_RE = re.compile(r"\p{L}[\p{L}\p{N}-]*", re.UNICODE)

In [3]:
# regex-шаблоны
RE_PERCENT_SIGN = re.compile(r"(?<!\d)\d{1,2}(?:[.,]\d)?\s*%")
RE_PERCENT_WORD = re.compile(r"\b\d{1,2}(?:[.,]\d)?\s*(?:проц|процент(?:а|ов)?)\b", re.IGNORECASE)
UNITS = ["мл","ml","l","л","г","гр","kg","кг","шт","уп","пак", "ш", "к"]
RE_VOLUME = re.compile(rf"\b\d+(?:[.,]\d+)?\s*(?:{'|'.join(UNITS)})\b", re.IGNORECASE)
RE_NUMBER = re.compile(r"\b\d+(?:[.,]\d+)?\b")

In [4]:
def _read_train(path: str) -> pd.DataFrame:
    return pd.read_csv(path, sep=';')

def parse_ann(s: str):
    try:
        return [(int(a[0]), int(a[1]), a[2]) for a in eval(s)]
    except Exception:
        return []

def build_fatty_lexicon(df: pd.DataFrame, top_k=200):
    mask = df['annotation'].map(parse_ann).map(lambda a: any(t.endswith('PERCENT') for *_,t in a))
    dfp = df[mask]
    cnt = Counter()
    for txt in dfp['sample']:
        for m in WORD_RE.finditer(str(txt).lower()):
            w = m.group(0)
            if len(w) > 2:
                cnt[w]+=1
    stop = {"для","и","в","на","по","без","со","из","от","до","за","процент","проц"}
    return [w for w,_ in cnt.most_common() if w not in stop][:top_k]

try:
    df_train = _read_train(TRAIN_PATH)
    FATTY_WORDS = build_fatty_lexicon(df_train)
except Exception:
    FATTY_WORDS = ["молоко","кефир","сливки","сметана","творог","сыр"]

In [5]:
def extract_explicit_numeric(text: str):
    ents=[]
    for rx in (RE_PERCENT_SIGN, RE_PERCENT_WORD):
        for m in rx.finditer(text):
            ents.append((m.start(),m.end(),'B-PERCENT'))
    for m in RE_VOLUME.finditer(text):
        ents.append((m.start(),m.end(),'B-VOLUME'))
    return sorted(ents)

PACK_WORDS=["бутыл","банка","пакет","упаков","рулон","лист","пачк","флакон"]

def infer_implicit_numeric(text: str, fatty_words=FATTY_WORDS):
    ents=[]
    for m in RE_NUMBER.finditer(text):
        s,e=m.span(); raw=text[s:e]
        if re.match(r"\d+%", raw): continue
        try: val=float(raw.replace(',', '.'))
        except: continue
        ctx=text[max(0,s-20):min(len(text),e+20)].lower()
        if val==0:
            ents.append((s,e,'B-PERCENT'))
        elif 1<=val<=99 and any(w in ctx for w in fatty_words):
            ents.append((s,e,'B-PERCENT'))
        elif val>=100 or any(w in ctx for w in PACK_WORDS):
            ents.append((s,e,'B-VOLUME'))
    return ents

def merge_overlapping_entities(entities: List[Tuple[int, int, str]]):
    """
    Схлопывает перекрывающиеся сущности одного типа.
    Если одна сущность полностью входит в другую, оставляем только большую.
    """
    if not entities:
        return []
    
    # Сортируем по начальной позиции
    entities = sorted(entities)
    result = []
    
    for start, end, label in entities:
        # Проверяем, есть ли перекрытие с последней добавленной сущностью
        if result:
            last_start, last_end, last_label = result[-1]
            
            # Если сущности одного типа и перекрываются
            if (label == last_label and 
                not (end <= last_start or start >= last_end)):
                
                # Если текущая сущность полностью входит в последнюю - пропускаем
                if start >= last_start and end <= last_end:
                    continue
                # Если последняя сущность полностью входит в текущую - заменяем
                elif last_start >= start and last_end <= end:
                    result[-1] = (start, end, label)
                    continue
                # Если частично перекрываются - берем объединение
                else:
                    result[-1] = (min(start, last_start), max(end, last_end), label)
                    continue
        
        result.append((start, end, label))
    
    return result

def preprocess_query(text: str):
    explicit=extract_explicit_numeric(text)
    implicit=infer_implicit_numeric(text)
    all_entities = sorted(explicit+implicit)
    return merge_overlapping_entities(all_entities)

In [6]:
import pandas as pd

df_examples = pd.read_csv("data/percent_example.csv", sep=";")
for idx, row in df_examples.iterrows():
    sample = row["sample"]
    annotation = row["annotation"]
    print(sample, annotation, ' => ', preprocess_query(sample), )

балтика 0 [(0, 7, 'B-BRAND'), (8, 9, 'B-PERCENT')]  =>  [(8, 9, 'B-PERCENT')]
кефир 1% [(0, 5, 'B-TYPE'), (6, 8, 'B-PERCENT')]  =>  [(6, 8, 'B-PERCENT')]
масло сливочное 72 [(0, 5, 'B-TYPE'), (6, 15, 'I-TYPE'), (16, 18, 'B-PERCENT')]  =>  [(16, 18, 'B-PERCENT')]
масло сливочное 82 [(0, 5, 'B-TYPE'), (6, 15, 'I-TYPE'), (16, 18, 'B-PERCENT')]  =>  [(16, 18, 'B-PERCENT')]
молоко 1 % [(0, 6, 'B-TYPE'), (7, 8, 'B-PERCENT'), (9, 10, 'I-PERCENT')]  =>  [(7, 10, 'B-PERCENT')]
молоко 3,2 [(0, 6, 'B-TYPE'), (7, 10, 'B-PERCENT')]  =>  [(7, 10, 'B-PERCENT')]
молоко ультрапастеризованное 3.2 [(0, 6, 'B-TYPE'), (7, 28, 'I-TYPE'), (29, 32, 'B-PERCENT')]  =>  []
мягкий творог 5% [(0, 6, 'B-TYPE'), (7, 13, 'I-TYPE'), (14, 16, 'B-PERCENT')]  =>  [(14, 16, 'B-PERCENT')]
сливки 10 [(0, 6, 'B-TYPE'), (7, 9, 'B-PERCENT')]  =>  [(7, 9, 'B-PERCENT')]
сливки 10% [(0, 6, 'B-TYPE'), (7, 10, 'B-PERCENT')]  =>  [(7, 10, 'B-PERCENT')]
сливки 20 [(0, 6, 'B-TYPE'), (7, 9, 'B-PERCENT')]  =>  [(7, 9, 'B-PERCENT')]
слив